## 深入浅出pandas-2

如果使用 pandas 做数据分析，那么`DataFrame`一定是被使用得最多的类型，它可以用来保存和处理异质的二维数据。这里所谓的“异质”是指`DataFrame`中每个列的数据类型不需要相同，这也是它区别于 NumPy 二维数组的地方。`DataFrame`提供了极为丰富的属性和方法，帮助我们实现对数据的重塑、清洗、预处理、透视、呈现等一系列操作。

### 创建DataFrame对象

#### 通过二维数组创建DataFrame对象

In [1]:
import numpy as np
import pandas as pd
scores = np.random.randint(60, 101, (5, 3))
courses = ['语文', '数学', '英语']
stu_ids = np.arange(1001, 1006) #[]
df1 = pd.DataFrame(data=scores, columns=courses, index=stu_ids)
df1

,语文,数学,英语
1001,75,74,86
1002,93,78,99
1003,70,91,94
1004,67,94,90
1005,63,99,67


#### 通过字典创建DataFrame对象

In [2]:
scores = {
    '语文': [62, 72, 93, 88, 93],
    '数学': [95, 65, 86, 66, 87],
    '英语': [66, 75, 82, 69, 82],
}
stu_ids = np.arange(1001, 1006)
df2 = pd.DataFrame(data=scores, index=stu_ids)
df2

,语文,数学,英语
1001,62,95,66
1002,72,65,75
1003,93,86,82
1004,88,66,69
1005,93,87,82


#### 读取CSV文件创建DataFrame对象

可以通过`pandas` 模块的`read_csv`函数来读取 CSV 文件，`read_csv`函数的参数非常多，下面介绍几个比较重要的参数。

- `sep` / `delimiter`：分隔符，默认是`,`。
- `header`：表头（列索引）的位置，默认值是`infer`，用第一行的内容作为表头（列索引）。
- `index_col`：用作行索引（标签）的列。
- `usecols`：需要加载的列，可以使用序号或者列名。
- `true_values` / `false_values`：哪些值被视为布尔值`True` / `False`。
- `skiprows`：通过行号、索引或函数指定需要跳过的行。
- `skipfooter`：要跳过的末尾行数。
- `nrows`：需要读取的行数。
- `na_values`：哪些值被视为空值。
- `iterator`：设置为`True`，函数返回迭代器对象。
- `chunksize`：配合上面的参数，设置每次迭代获取的数据体量。

In [3]:
df3 = pd.read_csv('data/2023年北京积分落户数据.csv', index_col='公示编号')
df3

,姓名,出生年月,单位名称,积分分值
公示编号,,,,
202300001,张浩,1977-02,北京首钢股份有限公司,140.05
202300002,冯云,1982-02,中国人民解放军空军二十三厂,134.29
202300003,王天东,1975-01,中建二局第三建筑工程有限公司,133.63
202300004,陈军,1976-07,中建二局第三建筑工程有限公司,133.29
202300005,樊海瑞,1981-06,中国民生银行股份有限公司,132.46
...,...,...,...,...
202305999,曹恰,1983-09,首都师范大学科德学院,109.92
202306000,罗佳,1981-05,厦门方胜众合企业服务有限公司海淀分公司,109.92
202306001,席盛代,1983-06,中国华能集团清洁能源技术研究院有限公司,109.92


> 数据来源于网络，侵删

#### 读取Excel工作表创建DataFrame对象

可以通过`pandas` 模块的`read_excel`函数来读取 Excel 文件，该函数与上面的`read_csv`非常类似，多了一个`sheet_name`参数来指定数据表的名称，但是不同于 CSV 文件，没有`sep`或`delimiter`这样的参数。假设有名为“2022年股票数据.xlsx”的 Excel 文件，里面有用股票代码命名的五个表单，分别是阿里巴巴（BABA）、百度（BIDU）、京东（JD）、亚马逊（AMZN）、甲骨文（ORCL）这五个公司2022年的股票数据，如果想加载亚马逊的股票数据，代码如下所示。


In [4]:
df4 = pd.read_excel('data/2022年股票数据.xlsx', sheet_name='AMZN', index_col='Date')
df4

,Open,High,Low,Close,Volume
Date,,,,,
2022-12-30,83.120,84.050,82.4700,84.000,62401194
2022-12-29,82.870,84.550,82.5500,84.180,54995895
2022-12-28,82.800,83.480,81.6900,81.820,58228575
2022-12-27,84.970,85.350,83.0000,83.040,57284035
2022-12-23,83.250,85.780,82.9344,85.250,57433655
...,...,...,...,...,...
2022-01-07,163.839,165.243,162.0310,162.554,46605900
2022-01-06,163.450,164.800,161.9370,163.254,51957780
2022-01-05,166.883,167.126,164.3570,164.357,64302720


#### 读取关系数据库二维表创建DataFrame对象

`pandas`模块的`read_sql`函数可以通过 SQL 语句从数据库中读取数据创建`DataFrame`对象，该函数的第二个参数代表了需要连接的数据库。对于 MySQL 数据库，我们可以通过`pymysql`或`mysqlclient`来创建数据库连接（需要提前安装好三方库），得到一个`Connection` 对象，而这个对象就是`read_sql`函数需要的第二个参数。

### 基本属性和方法

`DataFrame`对象的属性如下表所示。

| 属性名         | 说明                                |
| -------------- | ----------------------------------- |
| `at` / `iat`   | 通过标签获取`DataFrame`中的单个值。 |
| `columns`      | `DataFrame`对象列的索引             |
| `dtypes`       | `DataFrame`对象每一列的数据类型     |
| `empty`        | `DataFrame`对象是否为空             |
| `loc` / `iloc` | 通过标签获取`DataFrame`中的一组值。 |
| `ndim`         | `DataFrame`对象的维度               |
| `shape`        | `DataFrame`对象的形状（行数和列数） |
| `size`         | `DataFrame`对象中元素的个数         |
| `values`       | `DataFrame`对象的数据对应的二维数组 |

In [5]:
df4.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 251 entries, 2022-12-30 to 2022-01-03
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Open    251 non-null    float64
 1   High    251 non-null    float64
 2   Low     251 non-null    float64
 3   Close   251 non-null    float64
 4   Volume  251 non-null    int64  
dtypes: float64(4), int64(1)
memory usage: 11.8 KB


如果需要查看`DataFrame`的头部或尾部的数据，可以使用`head()`或`tail()`方法，这两个方法的默认参数是`5`，表示获取`DataFrame`最前面5行或最后面5行的数据，如下所示。

In [6]:
df4.head()

,Open,High,Low,Close,Volume
Date,,,,,
2022-12-30,83.12,84.05,82.4700,84.00,62401194
2022-12-29,82.87,84.55,82.5500,84.18,54995895
2022-12-28,82.80,83.48,81.6900,81.82,58228575
2022-12-27,84.97,85.35,83.0000,83.04,57284035
2022-12-23,83.25,85.78,82.9344,85.25,57433655


In [7]:
df4.tail()

,Open,High,Low,Close,Volume
Date,,,,,
2022-01-07,163.839,165.243,162.031,162.554,46605900
2022-01-06,163.450,164.800,161.937,163.254,51957780
2022-01-05,166.883,167.126,164.357,164.357,64302720
2022-01-04,170.438,171.400,166.349,167.522,70725160
2022-01-03,167.550,170.704,166.160,170.404,63869140


### 操作数据

#### 索引和切片

如果要获取`DataFrame`的某一列，例如取出上面`df4`的`Open`列，可以使用下面的两种方式。

In [8]:
df4.Open 
#或者 df4['Open']

Date
2022-12-30     83.120
2022-12-29     82.870
2022-12-28     82.800
2022-12-27     84.970
2022-12-23     83.250
               ...   
2022-01-07    163.839
2022-01-06    163.450
2022-01-05    166.883
2022-01-04    170.438
2022-01-03    167.550
Name: Open, Length: 251, dtype: float64

执行上面的代码可以发现，我们获得的是一个`Series`对象。事实上，`DataFrame`对象就是将多个`Series`对象组合到一起的结果。

如果要获取`DataFrame`的某一行，可以使用整数索引或我们设置的索引，例如取出'2022-12-30'(第0行)，代码如下所示。

In [9]:
df4.iloc[0] #df4[0] idx

Open            83.12
High            84.05
Low             82.47
Close           84.00
Volume    62401194.00
Name: 2022-12-30 00:00:00, dtype: float64

In [10]:
#或者
df4.loc['2022-12-30']

Open            83.12
High            84.05
Low             82.47
Close           84.00
Volume    62401194.00
Name: 2022-12-30 00:00:00, dtype: float64

通过执行上面的代码我们发现，单独取`DataFrame` 的某一行或某一列得到的都是`Series`对象。我们当然也可以通过花式索引来获取多个行或多个列的数据，花式索引的结果仍然是一个`DataFrame`对象。

获取多个列：

In [11]:
df4[['Open', 'High']]

,Open,High
Date,,
2022-12-30,83.120,84.050
2022-12-29,82.870,84.550
2022-12-28,82.800,83.480
2022-12-27,84.970,85.350
2022-12-23,83.250,85.780
...,...,...
2022-01-07,163.839,165.243
2022-01-06,163.450,164.800
2022-01-05,166.883,167.126


如果要获取或修改`DataFrame` 对象某个单元格的数据，需要同时指定行和列的索引。

In [12]:
df4['Open']['2022-12-30']

83.12

In [13]:
#或者
df4.loc['2022-12-30', 'Open']

83.12

In [14]:
df4.loc['2022-12-30', 'Open'] = 83.00

当然，我们也可以通过切片操作来获取多行多列，相信大家一定已经想到了这一点。

#### 数据筛选

上面我们提到了花式索引，相信大家已经联想到了布尔索引。跟`ndarray`和`Series`一样，我们可以通过布尔索引对`DataFrame`对象进行数据筛选。

In [15]:
df4[df4.Open > 160]

,Open,High,Low,Close,Volume
Date,,,,,
2022-04-06,161.651,162.200,157.254,158.756,79055760
2022-04-05,167.742,168.110,163.266,164.055,53728560
2022-04-04,164.125,168.394,163.206,168.346,50007960
2022-04-01,164.150,165.827,162.320,163.560,57089500
2022-03-31,166.445,166.495,162.954,162.998,59965780
2022-03-30,168.510,168.950,165.500,166.301,56167260
2022-03-29,170.384,170.832,167.868,169.315,66153600
2022-03-28,164.975,169.038,164.900,168.990,59853820
2022-03-25,164.000,165.368,162.250,164.773,49085500


当然，我们也可以组合多个条件来进行数据筛选。

In [16]:
df4[(df4.Open > 160) & (df4.High > 170)]

,Open,High,Low,Close,Volume
Date,,,,,
2022-03-29,170.384,170.832,167.868,169.315,66153600
2022-01-04,170.438,171.400,166.349,167.522,70725160
2022-01-03,167.550,170.704,166.160,170.404,63869140


除了使用布尔索引，`DataFrame`对象的`query`方法也可以实现数据筛选，`query`方法的参数是一个字符串，它代表了筛选数据使用的表达式，而且更符合 Python 程序员的使用习惯。下面我们使用`query`方法将上面的效果重新实现一遍，代码如下所示。


In [17]:
df4.query('Open > 160 and High > 170') # and or 

,Open,High,Low,Close,Volume
Date,,,,,
2022-03-29,170.384,170.832,167.868,169.315,66153600
2022-01-04,170.438,171.400,166.349,167.522,70725160
2022-01-03,167.550,170.704,166.160,170.404,63869140
